In [ ]:
import numpy as np
import gymnasium as gym
import matplotlib.pyplot as plt
import random


The code below visulizes the environment with random actions, the goal will be to create a policy that chooses actions. 

In [ ]:
env_render = gym.make("Taxi-v3", render_mode='human') # create environment with rendering 
state, _ = env_render.reset() # initiate the environment
done = False
i = 0
while not done:
    action = env_render.action_space.sample()  # Samples random actions from our environment. 
    new_state, reward, terminated, truncated, _ = env_render.step(action) # takes as step in our environment
    done = terminated or truncated # checks if we have reached our goal or timed out
    state = new_state # updates the current state.
    if i > 20: # End after 20 steps
        done = True
    i+=1
env_render.close()

## The Q-learning algorithm

In Q-learning we want to learn a look-up table that for every state and action has the future discounted reward. It is initialized with zeros. 

| State  | a1  | a2  | a3  | a4 |
|---|---|---|---|---|
| 1  | 0  | 0  | 0 | 0  |
| 2  | 0  | 0  | 0 | 0  |
| ...  | ...  | ...  | ... | ... |
| N  | 0  | 0  | 0 | 0  |

The training then updates this table to the correct values. It might look something like:

| State  | a1  | a2  | a3  | a4 |
|---|---|---|---|---|
| 1  | 0.1  | -5  | 1 | 0.4  |
| 2  | -1  | -2  | 0 | 4  |
| ...  | ...  | ...  | ... | ... |
| N  | 1  | 2  | -2 | 1  |

List of some terms in the feild of Q-learning:
* State - A state in our environment is a representation of all the information available to make a decision. At time step "t," we have the state "s_t." In the case of the taxi problem, there are 500 different states. Each state is an integer within the range of 0 to 500, and it represents: 
    * 25 different taxi locations
    * 5 passenger locations
    * 4 possible destinations
* Markov decision process (MDP) - Q-learning is an algorithm designed to solve problems involving finite discrete Markov decision processes (MDPs) https://en.wikipedia.org/wiki/Markov_decision_process. 
* Action - The actions is how we transition between states, at time step $t$ the action $a_t$ takes us to $s_{t+1}$. In this case we have 6 possible actions:
    * 0: Move south (down)
    * 1: Move north (up)
    * 2: Move east (right)
    * 3: Move west (left)
    * 4: Pickup passenger
    * 5: Drop off passenger
* Reward - In reinforcement learning, the system provides rewards and penalties to guide the learning process. When the actor successfully completes a task or makes a correct move, a reward $r$ is given. Conversely, penalties can be imposed for suboptimal or incorrect actions. These rewards and penalties serve as the means to influence and shape what the algorithm attempts to learn and optimize. The algorithm aims to maximize the cumulative rewards over time by making decisions that lead to higher overall rewards and fewer penalties.
* Q-table - The q table contains for every state (s) and action (a) the expected future discounted reward. Often written as Q(s, a), see above.
* Policy - How do we choose our action, Q(s, a) outputs how good a action is but not directly which action to choose. 
* Greedy policy - For a state choose the action that has the highest expected reward.
* Epsilon greedy policy - During training allow for exploration by having a probability of choosing a random action instead of greedy. Exploration is necessary for learning, without it the learning can get stuck in the same solution. 
* Episode - One espisode is the agient interacting with the environment from start to finish or termination i.e. a timeseries of states and actions.  

The q-table can be initated with zeros. It can be updated as:

$Q(s_t, a_t) = (1- \alpha)Q(s_t, a_t) + \alpha(r_t + \gamma \underbrace{\max}_{a} Q(s_{t+1}, a))$

where $\alpha$ is the learning rate and $\gamma$ is the dicount factor. For training, we want a policy that both explores new paths while at the same time exploits the current best solution. The solution is an epsilon greedy policy. There is more information at https://en.wikipedia.org/wiki/Q-learning. Also notice that we never update the Q values for the goal state, as the episode will terminate upon reaching the goal. The epsilon greedy policy:

$a \begin{cases} random & \textrm{With probability } \epsilon \\
                greedy &  \textrm{With probability } 1 - \epsilon \end{cases}$
                
Where $\epsilon$ is ussually decresing with traning, here is exponential decay. 

$ \epsilon = \epsilon_{\textrm{min}} + (\epsilon_{\textrm{max}} -\epsilon_{\textrm{min}}) e^{- \lambda i_e}$

where $\epsilon_{\textrm{min}}$ is the minimum value of $\epsilon$, i.e. we allways want some exploration, otherwise we wont learn anything new. $\epsilon_{\textrm{max}}$ usually set to $1$, how much exploration to start of with. $\lambda$ is the decay rate, how fast should we decreese the amount of exploration. $i_e$ is how many episodes we have trained on.

Next we can check how many state and actions our environment has. 

In [ ]:


env = gym.make("Taxi-v3")
num_states = env.observation_space.n
num_actions = env.action_space.n
print('Number of states', num_states, 'and number of actions', num_actions)

We define our hyper parameters. 

In [ ]:


# Hyperparameters
learning_rate = 0.1 # aplha in our q-learning update
discount_factor = 0.99 # gamma
epsilon = 1.0 # epsilon
epsilon_max = 1.0 # epsilon max
epsilon_min = 0.01 # epsilon min
epsilon_lambda = 0.01 # lambda
num_episodes = 5000

## Exersice 1.1: Q-learning 

Here the task is to implement the Q-learning algorithm. 

In [ ]:
# Initiate q-table

q_table = np.array(np.zeros([num_states, num_actions]))
print(q_table);

# Q-learning algorithm
for episode in range(num_episodes):
    state, info = env.reset()
    done = False
    total_reward = 0
    
    while not done:
        # Calculate action with exploration-exploitation trade-off
        
        # TODO
        if(random.uniform(0, 1) < epsilon):
            action = random.randint(0, num_actions - 1)
        else:
            action = np.argmax(q_table[state, :])
     
        # step 
        new_state, reward, terminated, truncated, _ = env.step(action)
        done = terminated or truncated
        
        # TODO
        # Q-value update
        total_reward = (1-learning_rate)*q_table[state, action] + learning_rate*(reward + discount_factor * np.argmax(q_table[new_state,:]))
        
        
        
        q_table[state, action] += total_reward 
        state = new_state

    # TODO
    # Decay exploration probability (epsilon)
    epsilon = epsilon_min + (epsilon_max - epsilon_min) * np.exp(-epsilon_lambda * episode)
    
    print(f"Episode: {episode+1}, Total Reward: {total_reward}")


## Exersice 1.2: Evaluate Q-learning 

Here implement greedy actions only with the learned Q-table. It is also possible to visualize the results. A coorectly implemented q-learning algorithm should get average reward of higher than 7.


In [ ]:
# Evaluate the trained policy
env_render = gym.make("Taxi-v3", render_mode='human') # env with renderig, if you want to visualize, but it is slow. 

total_rewards = []
num_evaluation_episodes = 100

for _ in range(num_evaluation_episodes):
    state, _ = env.reset()
    done = False
    total_reward = 0
    
    while not done:
        # TODO
        # Choose greedy action 
        
        # action = 
        new_state, reward, terminated, truncated, _ = env.step(action)
        #new_state, reward, terminated, truncated, _ = env_render.step(action) # uncomment for visualization

        done = terminated or truncated
        total_reward += reward
        state = new_state
    
    total_rewards.append(total_reward)

average_reward = np.mean(total_rewards)
print(f"Average Reward over {num_evaluation_episodes} evaluation episodes: {average_reward}")
env_render.close()


## Exersice 1.3: Q-learning question

Are there any limitations with Q-learning, what type of problems can it solve and what type of problems is it less suitable for?

# Deep Q learning

In this task, we'll tackle the challenge of balancing an inverted pendulum. What makes this problem intriguing is that it involves continuous input variables, making the traditional Q-learning approach with a finite table impractical. Instead, we'll use a neural network to approximate the Q-table, enabling us to efficiently handle continuous input while still producing discrete outputs.

First we will construct our neural network, this will be done in three steps. First we will define our activation function, then we will define a what a layer is and lastly we will construct the network.  

First let us go over the math of a neural network. They should be seen as general function approximator. A segment of neural network can be represented with a graph representation, see image below. 

<img src="imgs/NN_ps3.png" width="800"/>



### Naming
* $a^i_j$ is the output of neuron j in layer i.
* $b^i_j$ is the sum of the input to neuron j in layer i.
* $g()$ is the activation function of a neuron (the last layer does not have a activation funciton in this case)
* $w^i_{jk}$ is the weights of layer i going from neron k in layer i-1 to neuron j in layer i. 

### Forward propagation

A neural network can be seen as a function $y = f(x)$ where $x$ is an input vector and $y$ is an output vector. Forward propagation describes how we go from the input $x$ to the output $y$. Each layer can be caluculated in a sequence, where we start from the input. 

First each neuron $j$ in layer 1 calculated the sum of the input to that neuron:

$b^1_j = \sum_{k=1}^{len(x)}( w^1_{j,k}x_k) + w^1_{j,0}$ 

The output of the first layer is caluclated but using the activation funciton. 

$a^1_j = g(b^1_j)$

More generally for layer $i$ and with matrix multiplication we can write this as.

$b^i = w^i \begin{bmatrix} 1 \\ a^{i-1}\end{bmatrix}$

where $b^i$ is a vector that has the same dimension as number of neurons in layer $i$ and the 1 added to the input is for calculating the bias. 

$a^i = g(b^i)$

This is then caluculated sequentially until the last layer has been reashed. For the last layer without activation function we would have

$\hat{y} = a^i = b^i$
 
### Backward propagation

The way we train a network is by updating the weights $w$, they are the parameters we want to estimate to solve the problem. The way we are going to update the weights is through gradient descent. The gradients are calculated sequentially from the last layer to the first. The reason for this is that this allows us to leverage the chain rule, this saves us a lot of computational complexity. 

Calculating gradients:

For each parameter (weight) in the neural network we will calculate a gradient:
$\frac{\partial \mathcal{L}}{\partial w^i_{jk}}$ 
where $\mathcal{L(\mathbf{y}, \hat{\mathbf{y}})}$ is the loss function, which we want to minimize. A normal loss function could for example be the mean square error $\mathcal{L} = \frac{1}{n}\sum_{i=1}^n(y-\hat{y})^2$. The gradients can be caluclated with 

$\frac{\partial \mathcal{L}}{\partial w^i_{jk}} = \Delta^i_{j} a^{i-1}_k$

where $\Delta^i_j$ can be seen as the "precived" error for the input to neuron $j$ in layer $i$. It can be calculated with the chain rule. Let $n(i)$ be a function that returns the number of neuron of layer $i$.

$\Delta^i_j = \left( \sum_{r=1}^{n(i+1)} \Delta^{i+1}_r w^{i+1}_{rj} \right) \frac{\partial g(b^i_j)}{\partial b^i_j}$

For the first itteration there where is no $\Delta^{i+1}_r$ then they are calculated as:

$\Delta^i_j = \frac{\partial \mathcal{L(\mathbf{y}, \hat{\mathbf{y}})}}{\partial \hat{y}_j}$

This is under the assumption that there is no activation on the last layer. The nice thing is that the calculation of the vector $\Delta^i$ can be done as a matrix multiplication:

$\Delta^i = (\bar{w}^{i+1})^T \Delta^{i+1} \odot \nabla_{b^i} g(b^i)$

where $\odot$ is elementwise multiplication, $\bar{w}$ is the weight without bias and $\nabla_{b^i} g(b^i) = \begin{bmatrix} \frac{\partial g(b^i_1)}{\partial b^i_1} &...&\frac{\partial g(b^i_n)}{\partial b^i_n} \end{bmatrix}^T$.

We can then calculate the gradients for a layer with:

$\nabla_{w^i} \mathcal{L} = \Delta^i  (a^{i-1})^T$

We are then going to update the weights with:

$w^i \leftarrow w^i - \alpha(\nabla_{w^i} \mathcal{L} + \lambda w^i)$

where $\alpha$ is the learning rate and $\lambda$ is a regularization term (L2 regularization), we don't want the weight to become too large. The regularization also combats overfitting to some extent. We will come back to $\nabla_{w^i}$ later. 

For more on the chain rule see the exersice "NN_exersice" on canvas under the Exercises folder.  

## Exersice 2.1: ReLu

Here we will implement a class for the ReLu activation function. The activation function is what allows deep neural networks to express non-linearities. The class will only have two functions, forward(x) and diff(x). The forward function takes a vector or matrix and outputs the same shape where for every element we have applied the ReLu function. The diff function returns the partial derivative of the ReLu function. This will later be used in our neural network. 

Hint: np.where() is useful function

In [ ]:
class ReLu():
    def forward(self, x):
        # x is a numpy array and can have any shape. 
        # calculates the output of the ReLU function
        # TODO
        return = 
        
        
    def diff(self, x):
        # x is a numpy array and can have any shape.
        # calculates the partial differential of the ReLU function
        # TODO
        # return = 
        

In [ ]:
relu = ReLu()

input = np.array([[1.2],[-1.1], [0.0], [10.3]])
out = relu.forward(input)
ref = np.array([[1.2],[0.0], [0.0], [10.3]])
print('forward test: ', np.array_equal(out, ref))

diff_out = relu.diff(input)
diff_ref = np.array([[1.],[0.0], [0.0], [1]])
print('diff test: ', np.array_equal(diff_out, diff_ref))
 


## Exersice 2.2: Layer

A dense neural network is constructed by layers of neurons, here we will define a class to represent our layers. It will contain the weights and activation function for the layer. The layer class will contain two functions, forward and backward.

Hint: it is a good practice to return a copy of np arrays or matrices as otherwise modifying the output can modify the internally saved matrix as it is the same object, these problems are not easy to debug. For a matrix A, then return A.copy(). 


In [ ]:
class Layer(object):
    def __init__(self, num_in, num_out, activation_func=None):
        self.weights = np.random.rand(num_out, num_in+1)-0.5 # (to layer, from layer) 
        self.gradients = np.zeros((num_out, num_in+1))
        self.a_input = None  # placeholder for saving the input to the forward function
        self.b = None  # placeholder for saving the value right before the activation function
        self.activation_func = activation_func  # class instance of activation function 
        
    def forward(self, x):
        # TODO
        # x is a standing vector
        # Add a 1 for the bias term on the input, and save this in the self.a_input, this will be used later in backward()
        # Calculate b, the summation in each neuron. Can be done as a matrix multiplication, save in self.b. 
        # Check if the activation function is None, otherwise run the forward of the activation function. 
        # return the output of the layer
        
        
    
    def backward(self, pre_delta, pre_w=None):
        # TODO 
        # pre_delta is from layer i+1, pre_w is the weights from layer i+1.
        # First check it pre_w is None, it is none for the last layer (output layer).
        # Calculate this layers delta 
        # Caluclate this layers gradients and save to self.gradients (do not use the gradients to update the weights yet)
        # return delta from this layer 
        # Hint: see calculations above under Deep Q Learning
        
        


# Reinforcement learning and deep-learning

## General outline for this assingment 

* Q-learning, openai taxi problem 
* Deep Q-learning, control inverted pendelum
* Image classification with pytorch for MNIST dataset

This assignment combines the elements of comprehension and implementation. You will be responsible for coding in some sections, while in others, we will provide you with code to study.

### Q-Learning

In this task, you will employ Q-learning to tackle decision-making in a grid world game. This will provide you with a foundational grasp of Q-learning and the principles of reinforcement learning.

To set up the environment for the first two tasks, you'll need to install the required software. https://pypi.org/project/gymnasium/

pip install gymnasium==0.29.1


### Deep Q-Learning

Deep Q-learning combines neural networks with Q-learning. In this task, you're embarking on a rather ambitious journey. Your goal is to construct a neural network from the ground up and we will show how to apply it to a reinforcement learning problem. This endeavor aims to lay a solid foundation for your comprehension of neural networks and the inherent challenges they pose. This may well be one of the very few opportunities you'll have to create such a network from scratch, granting you insight into what transpires behind the scenes. Following this, you'll likely rely on libraries like PyTorch or TensorFlow, which abstract much of the mathematics. 

The core concept behind deep Q-learning involves approximating the Q-table with a neural network. However, as you'll soon discover, this endeavor is far from straightforward.


### CNN + dense with pytorch

To wrap up, you will employ PyTorch to tackle an image classification problem, specifically the classification of handwritten numbers. In this segment, we will demonstrate how PyTorch can be used to construct neural networks. Your objective is to enhance the accuracy of this model to reach a minimum of 98%.

You can easily install pytorch through (it is enough with just cpu support, if you have an nvidia graphics card you can try the cuda installation), https://pytorch.org/

I'm running pytorch version 2.0.1, newer versions should hopefully also work.

I have also only tested this assignment with numpy version 1.xx

# Q-Learning

In reinforcement learning, the primary aim is to maximize future rewards. While this may not seem significantly different from supervised learning, the implications are profound. Consider the scenario where you want an AI to navigate a maze: at each time step, the AI can make a move in any valid direction. If you were to approach this as a regression problem, akin to supervised learning, you would need to know the precise solution for every step, which becomes infeasible for complex problems. Reinforcement learning simplifies the human side of the problem by using rewards. For instance, you can reward the AI only when it successfully solves the maze and allow the AI to explore freely. You don't instruct it on how to solve the maze directly but instead provide a reward for completing the task. The AI will then determine the most effective way to maximize the reward over the entire episode. 

If you want a robot to learn how to walk, you would provide rewards for moving in the correct direction and penalties if it falls, but you wouldn't explicitly instruct it on how to walk. This is the advantage of reinforcement learning. 

Q-learning represents one of the simplest versions of reinforcement learning.   

In this task, we will employ Q-learning to tackle the taxi problem, which is further described in https://gymnasium.farama.org/environments/toy_text/taxi/ and visualization is provided a few cells down. Short summary: 


>There are four designated pick-up and drop-off locations (Red, Green, Yellow and Blue) in the 5x5 grid world. The taxi starts off at a random square and the passenger at one of the designated locations.
The goal is move the taxi to the passenger’s location, pick up the passenger, move to the passenger’s desired destination, and drop off the passenger. Once the passenger is dropped off, the episode ends.
The player receives positive rewards for successfully dropping-off the passenger at the correct location. Negative rewards for incorrect attempts to pick-up/drop-off passenger and for each step where another reward is not received.

Let's start by importing the necessary libraries

The code below visulizes the environment with random actions, the goal will be to create a policy that chooses actions. 

## The Q-learning algorithm

In Q-learning we want to learn a look-up table that for every state and action has the future discounted reward. It is initialized with zeros. 

| State  | a1  | a2  | a3  | a4 |
|---|---|---|---|---|
| 1  | 0  | 0  | 0 | 0  |
| 2  | 0  | 0  | 0 | 0  |
| ...  | ...  | ...  | ... | ... |
| N  | 0  | 0  | 0 | 0  |

The training then updates this table to the correct values. It might look something like:

| State  | a1  | a2  | a3  | a4 |
|---|---|---|---|---|
| 1  | 0.1  | -5  | 1 | 0.4  |
| 2  | -1  | -2  | 0 | 4  |
| ...  | ...  | ...  | ... | ... |
| N  | 1  | 2  | -2 | 1  |

List of some terms in the feild of Q-learning:
* State - A state in our environment is a representation of all the information available to make a decision. At time step "t," we have the state "s_t." In the case of the taxi problem, there are 500 different states. Each state is an integer within the range of 0 to 500, and it represents: 
    * 25 different taxi locations
    * 5 passenger locations
    * 4 possible destinations
* Markov decision process (MDP) - Q-learning is an algorithm designed to solve problems involving finite discrete Markov decision processes (MDPs) https://en.wikipedia.org/wiki/Markov_decision_process. 
* Action - The actions is how we transition between states, at time step $t$ the action $a_t$ takes us to $s_{t+1}$. In this case we have 6 possible actions:
    * 0: Move south (down)
    * 1: Move north (up)
    * 2: Move east (right)
    * 3: Move west (left)
    * 4: Pickup passenger
    * 5: Drop off passenger
* Reward - In reinforcement learning, the system provides rewards and penalties to guide the learning process. When the actor successfully completes a task or makes a correct move, a reward $r$ is given. Conversely, penalties can be imposed for suboptimal or incorrect actions. These rewards and penalties serve as the means to influence and shape what the algorithm attempts to learn and optimize. The algorithm aims to maximize the cumulative rewards over time by making decisions that lead to higher overall rewards and fewer penalties.
* Q-table - The q table contains for every state (s) and action (a) the expected future discounted reward. Often written as Q(s, a), see above.
* Policy - How do we choose our action, Q(s, a) outputs how good a action is but not directly which action to choose. 
* Greedy policy - For a state choose the action that has the highest expected reward.
* Epsilon greedy policy - During training allow for exploration by having a probability of choosing a random action instead of greedy. Exploration is necessary for learning, without it the learning can get stuck in the same solution. 
* Episode - One espisode is the agient interacting with the environment from start to finish or termination i.e. a timeseries of states and actions.  

The q-table can be initated with zeros. It can be updated as:

$Q(s_t, a_t) = (1- \alpha)Q(s_t, a_t) + \alpha(r_t + \gamma \underbrace{\max}_{a} Q(s_{t+1}, a))$

where $\alpha$ is the learning rate and $\gamma$ is the dicount factor. For training, we want a policy that both explores new paths while at the same time exploits the current best solution. The solution is an epsilon greedy policy. There is more information at https://en.wikipedia.org/wiki/Q-learning. Also notice that we never update the Q values for the goal state, as the episode will terminate upon reaching the goal. The epsilon greedy policy:

$a \begin{cases} random & \textrm{With probability } \epsilon \\
                greedy &  \textrm{With probability } 1 - \epsilon \end{cases}$
                
Where $\epsilon$ is ussually decresing with traning, here is exponential decay. 

$ \epsilon = \epsilon_{\textrm{min}} + (\epsilon_{\textrm{max}} -\epsilon_{\textrm{min}}) e^{- \lambda i_e}$

where $\epsilon_{\textrm{min}}$ is the minimum value of $\epsilon$, i.e. we allways want some exploration, otherwise we wont learn anything new. $\epsilon_{\textrm{max}}$ usually set to $1$, how much exploration to start of with. $\lambda$ is the decay rate, how fast should we decreese the amount of exploration. $i_e$ is how many episodes we have trained on.

Next we can check how many state and actions our environment has. 

We define our hyper parameters. 

## Exersice 1.1: Q-learning 

Here the task is to implement the Q-learning algorithm. 

## Exersice 1.2: Evaluate Q-learning 

Here implement greedy actions only with the learned Q-table. It is also possible to visualize the results. A coorectly implemented q-learning algorithm should get average reward of higher than 7.


## Exersice 1.3: Q-learning question

Are there any limitations with Q-learning, what type of problems can it solve and what type of problems is it less suitable for?

# Deep Q learning

In this task, we'll tackle the challenge of balancing an inverted pendulum. What makes this problem intriguing is that it involves continuous input variables, making the traditional Q-learning approach with a finite table impractical. Instead, we'll use a neural network to approximate the Q-table, enabling us to efficiently handle continuous input while still producing discrete outputs.

First we will construct our neural network, this will be done in three steps. First we will define our activation function, then we will define a what a layer is and lastly we will construct the network.  

First let us go over the math of a neural network. They should be seen as general function approximator. A segment of neural network can be represented with a graph representation, see image below. 

<img src="imgs/NN_ps3.png" width="800"/>



### Naming
* $a^i_j$ is the output of neuron j in layer i.
* $b^i_j$ is the sum of the input to neuron j in layer i.
* $g()$ is the activation function of a neuron (the last layer does not have a activation funciton in this case)
* $w^i_{jk}$ is the weights of layer i going from neron k in layer i-1 to neuron j in layer i. 

### Forward propagation

A neural network can be seen as a function $y = f(x)$ where $x$ is an input vector and $y$ is an output vector. Forward propagation describes how we go from the input $x$ to the output $y$. Each layer can be caluculated in a sequence, where we start from the input. 

First each neuron $j$ in layer 1 calculated the sum of the input to that neuron:

$b^1_j = \sum_{k=1}^{len(x)}( w^1_{j,k}x_k) + w^1_{j,0}$ 

The output of the first layer is caluclated but using the activation funciton. 

$a^1_j = g(b^1_j)$

More generally for layer $i$ and with matrix multiplication we can write this as.

$b^i = w^i \begin{bmatrix} 1 \\ a^{i-1}\end{bmatrix}$

where $b^i$ is a vector that has the same dimension as number of neurons in layer $i$ and the 1 added to the input is for calculating the bias. 

$a^i = g(b^i)$

This is then caluculated sequentially until the last layer has been reashed. For the last layer without activation function we would have

$\hat{y} = a^i = b^i$
 
### Backward propagation

The way we train a network is by updating the weights $w$, they are the parameters we want to estimate to solve the problem. The way we are going to update the weights is through gradient descent. The gradients are calculated sequentially from the last layer to the first. The reason for this is that this allows us to leverage the chain rule, this saves us a lot of computational complexity. 

Calculating gradients:

For each parameter (weight) in the neural network we will calculate a gradient:
$\frac{\partial \mathcal{L}}{\partial w^i_{jk}}$ 
where $\mathcal{L(\mathbf{y}, \hat{\mathbf{y}})}$ is the loss function, which we want to minimize. A normal loss function could for example be the mean square error $\mathcal{L} = \frac{1}{n}\sum_{i=1}^n(y-\hat{y})^2$. The gradients can be caluclated with 

$\frac{\partial \mathcal{L}}{\partial w^i_{jk}} = \Delta^i_{j} a^{i-1}_k$

where $\Delta^i_j$ can be seen as the "precived" error for the input to neuron $j$ in layer $i$. It can be calculated with the chain rule. Let $n(i)$ be a function that returns the number of neuron of layer $i$.

$\Delta^i_j = \left( \sum_{r=1}^{n(i+1)} \Delta^{i+1}_r w^{i+1}_{rj} \right) \frac{\partial g(b^i_j)}{\partial b^i_j}$

For the first itteration there where is no $\Delta^{i+1}_r$ then they are calculated as:

$\Delta^i_j = \frac{\partial \mathcal{L(\mathbf{y}, \hat{\mathbf{y}})}}{\partial \hat{y}_j}$

This is under the assumption that there is no activation on the last layer. The nice thing is that the calculation of the vector $\Delta^i$ can be done as a matrix multiplication:

$\Delta^i = (\bar{w}^{i+1})^T \Delta^{i+1} \odot \nabla_{b^i} g(b^i)$

where $\odot$ is elementwise multiplication, $\bar{w}$ is the weight without bias and $\nabla_{b^i} g(b^i) = \begin{bmatrix} \frac{\partial g(b^i_1)}{\partial b^i_1} &...&\frac{\partial g(b^i_n)}{\partial b^i_n} \end{bmatrix}^T$.

We can then calculate the gradients for a layer with:

$\nabla_{w^i} \mathcal{L} = \Delta^i  (a^{i-1})^T$

We are then going to update the weights with:

$w^i \leftarrow w^i - \alpha(\nabla_{w^i} \mathcal{L} + \lambda w^i)$

where $\alpha$ is the learning rate and $\lambda$ is a regularization term (L2 regularization), we don't want the weight to become too large. The regularization also combats overfitting to some extent. We will come back to $\nabla_{w^i}$ later. 

For more on the chain rule see the exersice "NN_exersice" on canvas under the Exercises folder.  

In [ ]:
class ReLu():
    def forward(self, x):
        # x is a numpy array and can have any shape. 
        # calculates the output of the ReLU function
        # TODO
        return = 
        
        
    def diff(self, x):
        # x is a numpy array and can have any shape.
        # calculates the partial differential of the ReLU function
        # TODO
        # return = 
        

## Exersice 2.2: Layer

A dense neural network is constructed by layers of neurons, here we will define a class to represent our layers. It will contain the weights and activation function for the layer. The layer class will contain two functions, forward and backward.

Hint: it is a good practice to return a copy of np arrays or matrices as otherwise modifying the output can modify the internally saved matrix as it is the same object, these problems are not easy to debug. For a matrix A, then return A.copy(). 


### Test your layer

## Exercise 2.3: Define the neural network
We will create a neural network that has two hidden layers with 64 neurons each with ReLU activation function and the output layer has the same size as the number of outputs but with no activation function. 

## Test supervised learning to see if the network performes as expeced. 
Here we want to test the neural network on a simple supervised problem, it should learn the output of two functions. Does the y_est follow the true y? You should see that they follow each other fairly well, usually within 0.1 difference. 

## Training Deep Q-learning

The idea for this section is that we will walk you through the code and the process and then you will answer some theoretical questions.

Now, we have a neural network, and we aim to employ it to balance an inverted pendulum on a cart. This problem involves continuous input space, with four inputs, and a discrete output space, consisting of two possible actions (left or right). The reward function for this task is straightforward - we receive a reward for every step taken, with the objective being to maintain balance. The task is considered unsuccessful if the pendulum leans too far or the cart moves excessively to the side. You can find more details about this problem at https://gymnasium.farama.org/environments/classic_control/cart_pole/.

The challenge with this problem is that the continuous input space makes it impractical to use standard Q-learning, as the Q-table would become excessively large. To address this, we'll replace the Q-table with a neural network. However, as we will demonstrate, this transition is not without its challenges. 

The primary challenge in deep Q-learning lies in training stability, as the training process tends to be noisy, and many issues can arise. To enhance stability, we introduce three key components: target networks, experience replay, and batch gradient descent. However, before delving into those specifics, let's discuss the fundamental principle of replacing the Q-table with a neural network.

In Q-learning, we maintain a table of future expected rewards, and for a given state and action, we can obtain the value, denoted as $Q(s, a)$. Our objective is to approximate this Q-value using a neural network, denoted as $f$, which takes the state as input and produces a value for each possible action, i.e., $\hat{q} = f(s)$, where $\hat{q}$ is a vector. The training loop remains similar to traditional Q-learning, but in this case, we formulate a loss function. 

$\mathcal{L}(s_t, a) = (q(s_t)_a -\hat{q}(s_t)_a)^2$

were 

$q(s_t)_a = r + \gamma \underbrace{\max}_{a} \hat{q}(s_{t+1})$ 

and $\hat{q}(s_t)_a$ is the output of the network for action $a$ and state $s$, and $\hat{q}(s_{t+1})$ is for the next state. This will make the network converge towarde the Q-table, atleast theoretically. The problem is that this is prone to instabilities. We can no longer initiate the network to predict zeros initially, so the initial predictions of the future reward will be completely wrong. Every time we update the weights we will also change $\underbrace{\max}_{a} \hat{q}(s_{t+1})$, which means that the value the network tries to converge to is allways changing, and if it varies to fast instabilities will arrise. One other of the downsides of using gradient decent is that it can easily forget thing that it trained on a long time ago. The methods below attempts to minimize these issues.     


### Target network 
The idea here is to make $\underbrace{\max}_{a} \hat{q}(s_{t+1})$ a slow varying target to increase stability. We introdue a second network called the target network. So we have two netorks $\hat{q} = f(s)$ and $\hat{q}_T = f_T(s)$, where the second is the target network. We use the second network to estimate the future estimated reward.  

$q(s_t, a) = r + \gamma \underbrace{\max}_{a} \hat{q}_T(s_{t+1}, a)$ 

The idea is then to update the target network slowly towards the first network, the weights of the target network $w_T$ can then be updated with 

$w_T = w_T + \alpha_T (w - w_T)$

where $\alpha_T$ is the target update rate. This adds stability to the traning as the future expected reward is more steady and slow changing. 

### Experiance replay

Next up is experience replay, because of gradient descent we don't want to train on samples that are too close to each other as there is a risk that the network will forget things it has learned. The idea is then in the training loop to not directly train on the observed data but instead save it in a buffer/queue. For training we sample for the queue, this means that the network might train on old data or new data. This minimizes that it forgets or that a lot of samples in sequence are too similar. 

### Batch gradient decent 

To furter imporove stability we sample multiple timesteps from our memory buffer, then we caluclate the gradients for each of them and take the average of the gradients. It is then those average graidentes we use as $\nabla_{w^i}$ to update the weights, this minimize noise as each update of the weights is based on multiple data samples. 

We will now show how these things can be implemented in code. 

## Hyper parameters

Neural networks can have quite a few hyper parameters and it can be quite tricky to tune them. I have provided some hypter parameters that work, but feel free to tune them if desiered.  

## Training function

Here we define our training function, this is what will be called for every step in our trianing loop. This part is responsible for sampling a batch from the memory buffer and calculating the gradients for the batch, it is also here we have our reward function and update our network. Lastly we will update the weights of the target network. 

##  Training loop

Here is the actual training loop that runs the simulation. This part should look similar to the q-learning training loop at the beginning of the assignment. The difference is the call of the training function and appending to the memory buffer. Because of the unstable nature of deep Q-learning, you typically save the network that performed the best during training. It can sometimes take more than 100 episodes before you see any noticeable learning.

Even with all the additions of the target network, batch gradient decent and experience replay, the training can still be unstable. It is therefore common in reinforcement learning to always save the best-performing network. 

## Exercise 2.4: Epsilon greedy - deep learning
Fill code for action and update of epsilon.

## Plot the learning curve
Below we plot the total reward againt the episodes, you can see that the learning is noisy and quite often it completly forgets everything after 300-400 episodes. The highest score is 500 and it is usually posible to reach it. 

## Test the learned policy

## Exersice 2.5: 

1. What are the similarities between Q-learning and deep Q-learning?
2. What are the differences and what implications does these difference have? Are Q-learning and deep Q-learning equally good at the same tasks or are they suited for different problems? 
3. Are there any difficulties with deep Q-learning?
4. What conclusion can you draw from the the learning curve?

# CNN and pytorch

## Exersice 3.1

Now that we've gained a solid understanding of how neural networks function and how they are trained, we'll shift our focus to tackling an image classification problem. In this instance, we will utilize ready-made packages and libraries to streamline the process.

Debugging neural networks is not an easy task! If you're looking for a helpful resource, the University of Amsterdam's course on debugging neural networks might provide valuable insights and techniques to address common challenges. https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/guide3/Debugging_PyTorch.html

The starting initial code is based on https://pytorch.org/tutorials/beginner/basics/quickstart_tutorial.html

We are going to use the MNIST data set, which containts images of handwritten numbers and your task is to classify the written numbers from the images. 

Your task is to modify the code below to increase the accuracy to atleast 98%. There will hints written throughout below. Then in task 3.2 you will describe the changes you made and try to motivate why you made them.    


In [ ]:


import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torch.nn.functional as F

In [ ]:


# Download training data from open datasets.
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

# visualize the data
num_images_to_plot = 9
fig, axes = plt.subplots(3, 3, figsize=(8, 8))

for i in range(num_images_to_plot):
    image, label = training_data[i]
    ax = axes[i // 3, i % 3]
    ax.imshow(image.squeeze(), cmap='gray')
    ax.set_title(f"Label: {label}")
    ax.axis('off')

plt.show()


In [ ]:


# Batch size, this is something you can tune, but the current value is decent.  
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break


## Define the neural network

Here there is alot of things that can be tuned, I do encurage to add convolutional layers. While it is not neccessary to achive the accuracy, the convolutional layers are very good at images. For example a convolution layer can be defnined as following

self.conv1 = nn.Conv2d(1, 3, 5)

where the first input is the number of in channels. For a grey image there is only one channel, for an rgb image there is 3 channels. The second input is the number of out channels, you can see it as parrallel filters. The last input is the filter size. More infromation exists at https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html. The thing to keep in mind is the dimensions out of convolution. In this case the inpout is 1x28x28 and the output 3x24x24.

In the forward funciton it can be used as

x =  F.relu(self.conv1(x))

And lets say you have defined a linear (dense) layer 

self.fd1 = nn.Linear(3 * 24 * 24, 120)

They can be combined in the forward function as:

x =  F.relu(self.conv1(x))

x = torch.flatten(x, 1) # this flattens the output to a vector

x =  F.relu(self.fd1(x))


A lot of things can be modified here, such as number of layers, the size of the layers, the type of layers. 


In [ ]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# TODO, modify 
# Define the neural network model
class NeuralNetwork_pytorch(nn.Module):
    def __init__(self):
        super().__init__()
        self.fd1 = nn.Linear(28*28, 512) # dense layers (input, output), output is the number of neurons for the layer
        self.fd2 = nn.Linear(512, 512)
        self.fd3 = nn.Linear(512, 10)

    def forward(self, x):
        x = torch.flatten(x, 1) 
        x = F.relu(self.fd1(x))
        x = F.relu(self.fd2(x))
        x = self.fd3(x)
        return x




model = NeuralNetwork_pytorch().to(device)
print(model)

In [ ]:
# Our traning function, nothing to modify here
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % (100*int(64/batch_size)) == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [ ]:
# our test function, nothing to modify here.
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [ ]:



# The training loop, here you can change the optimizer, the opimizer is how you update the weights. SGD does 
# gradient decent. Adam optimizer adds some extra steps and regularization. 

epochs = 5 # try to not change the number of epochs, we want something that learns fast. 
           # One epoch trains through all the data. 
    
model = NeuralNetwork_pytorch().to(device)
loss_fn = nn.CrossEntropyLoss() # you can change the loss function 

# you can change the optimizer. The optimizer is how you updates the weights and there are differnet ways to do that.
# For more info see https://uvadlc-notebooks.readthedocs.io/en/latest/tutorial_notebooks/tutorial4/Optimization_and_Initialization.html#Optimization
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2)
#optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

## Exersice 3.2: modifications

What modifications did you do and what improvments did you see from them. Can you reason to why they worked?

Text answer: 